# Least Squares Monte Carlo (Longstaff and Schwartz RFS 2001)

This notebook accompanies the paper "Benchmarking Machine Learning Software and Hardware for Quantitative Economics" and illustrates the Least Squares Option Pricing method proposed in Longstaff Schwartz (RFS 2001).

## Model Description

The discrete time approximation of an American option is the so-called Bermuda option, where the holder has the option to exercise the contract in a finite number of dates $0<t_1<t_2<...<t_{K-1}<t_K=T.$

Under the assumption of no arbitrage, the put option price $V_0$ is the solution of the following optimal stopping problem
\begin{align}
V_{0} = \sup_{\tau\in\mathcal{T}_0} \mathbb{E}^\mathbb{Q}\left[f(\tau, S_\tau)|\mathcal{F}_{0}\right],
\end{align}
where $S_\tau$ is the underlying asset, $f(\cdot,\cdot)$ is the discounted payoff function, the expectation is taken under the risk-neutral measure $\mathbb{Q}$, $\mathcal{F}_{0}$ represents the information set at the initial time, and the stopping time $\tau$ belongs to the class of all $\{0,...,T\}$-valued stopping times, represented by $\mathcal{T}_0$.

At the exercise date $t_i$, the continuation value $q_{t_i}$ satisfies
\begin{align}\label{eq:continuationvalue}
q_{t_i} = \sup_{\tau\in\mathcal{T}_{t_i}} \mathbb{E}^\mathbb{Q}\left[f(\tau,S_\tau)|\mathcal{F}_{t_i}\right],
\end{align}
where $\mathcal{F}_{t_i}$ is the information set at time $t_i$ and $\mathcal{T}_{t_i}$ is the class of all $\{t_{i+1},...,T\}$-valued stopping times.  The continuation values are determined by the recursive equations


\begin{align*}
q_{t_{i}} =  \mathbb{E}^\mathbb{Q}\left[\max\left\{f(t_{i+1},S_{t_{i+1}}), q_{t_{i+1}}\right\} |\mathbb{F}_{t_i}\right],\, i\in\{0,1,...,K-1\},
\end{align*}

with terminal condition $q_T = 0$.

Longstaff Schwartz (RFS 2001) use a linear combination of orthonormal basis functions to approximate the expectation above.
Starting at time $t_{K-1}$, the continuation value is approximated by
\begin{align}\label{eq:qt}
q_{t_{K-1}} = \sum_{j=0}^M a_j p_j(S_{t_{K-1}}),
\end{align}
where $a_j\in \mathbb{R}$ are the regression coefficients, $p_j(\cdot)$ are the polynomial basis, and  $M$ represents the degree of the polynomial basis.

The coefficients are determined by solving the least squares problem of minimizing the distance between the approximate option price and realized payoffs one period ahead.
To alleviate the problem of multicollinearity of the regressors, we solve the ordinary least squares problem with ridge regression using a $L_2$ penalty $\lambda=100$, and repeat this procedure until the first exercise date.


In [1]:
# Import libraries
import numpy as np
import tensorflow as tf

Spot = tf.Variable(36.)  # Stock spot price
σ = tf.Variable(0.2)     # Stoch instantaneous volatility
K = tf.Variable(40.)     # Strike price
r = tf.Variable(0.06)    # Instantaneous risk-free rate
T = 1                    # Maturity
order = 25               # Order of the polynomial approximations
n = 100000               # Number of independent paths   
m = 10                   # Number of time steps
Δt = T / m

## Auxiliary Functions
To make the code cleanear and easier to inspect, we broke down the LSMC algorithm
into 5 steps, implemented by different functions. 

In [2]:
def chebyshev_basis(x, k):
    """
    Creates a matrix with the Chebyshev polynomials of first kind up to
    the degree k, evaluated at the x. The function returns a matrix where
    the n-th column is T_n(x), for 0 < n.
    """
    B = {}
    B[0] = tf.ones_like(x)
    B[1] = x
    for n in range(2, k):
        B[n] = 2 * x * B[n - 1] - B[n - 2]

    return tf.stack(list(B.values()), axis=1)


def ridge_regression(X, Y, λ=100):
    """
    Performs a ridge regression with $L_2$ penalty $\lambda$.
    That is, given a matrix $X$ and a vector $Y$, it solves the least squares problem:

            \beta = argmin_Z ||X Z - Y||^2 + \lambda ||Z||^2  
        

            Returns: \hat{Y} = X \beta
    """
    
    β = tf.linalg.lstsq(X, tf.reshape(Y, [-1, 1]), l2_regularizer=100)
    return tf.squeeze(X @ β)

def first_one(x):
    """
    The 'first_one' function receives a matrix of payoffs and identifies the
    time period where the put option is exercised for each of the n simulated paths.
    """
    original = x
    x = tf.cast(x > 0, x.dtype)
    n_columns = x.shape.as_list()[1]
    batch_size = x.shape.as_list()[0]
    x_not = 1 - x
    sum_x = tf.minimum(tf.cumprod(x_not, axis=1), 1.)
    ones = tf.ones([batch_size, 1])
    lag = sum_x[:, :(n_columns - 1)]
    lag = tf.concat([ones, lag], axis=1)
    return original * (lag * x)


def scale(x):
    """
    Linearly scales a vector x to the domain [-1, 1], as required by
    standard Chebyshev polynomials.
    """

    xmin = tf.reduce_min(x)
    xmax = tf.reduce_max(x)
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


def advance(S):
    """
    Simulates the evolution of the stock price for one step and n paths.
    """
    dB = np.sqrt(Δt) * tf.random_normal(shape=[n])
    out = S + r * S * Δt + σ * S * dB
    return out

## Main Code
Here we define the computation graph. By default, TensorFlow executes in graph mode, so
    none of these intermediary operations are executed in this block. This eliminates overhead.

In [3]:
# We will store all relevant stochastic processes in dictionaries. For instance, the
# stock price at time t=5 is stored at S[0.5].
S = {0.: Spot * tf.ones(n)}

# Simulate the stock price evolution from t=0 to t=T
t_span = np.round(np.arange(Δt, T + Δt, Δt), 6)
for t in t_span:
    t_previous = np.round(t - Δt, 6)
    S[t] = advance(S[t_previous])

# time discount factor
discount = tf.exp(-r * Δt)

# cashflows IF the option is exercised. If the stoch price St is less than
# it's strike price K,  the cashflow is K - St if the option is exercised.
# Otherwise it is 0
cashflow = {t: tf.maximum(0., K - S[t]) for t in t_span}

# Recursion 
value = {T: cashflow[T] * discount}
continuation_value = {T: tf.zeros(n)}

for t in t_span[::-1][1:]:
    t_next = np.round(t + Δt, 6)

    basis = chebyshev_basis(scale(S[t]), order)
    continuation_value[t] = ridge_regression(basis, value[t_next])
    value[t] = discount * tf.where(cashflow[t] > continuation_value[t],
                                   cashflow[t],
                                   value[t_next])

# If the continuation value is larger than the cashflow (if the option is
# exercised), then it is optimal not to exercise. The payoff in that case is zero.
payoff = {t: tf.where(continuation_value[t] > cashflow[t],
                      tf.zeros(n),
                      cashflow[t]) for t in t_span}

# Stack the payoff into a n x m matrix (paths x periods)
payoff = tf.stack(list(payoff.values()), axis=1)

# Select only the first payoff: once you exercise the option you
# don't get any further payoff.
payoff = first_one(payoff)

# present value of payoffs
discounted_payoff = {i: payoff[:, i] * tf.exp(-r * i * Δt) for i in range(m)}

# The price is the expected value of the payoffs
price = tf.reduce_mean(tf.add_n(list(discounted_payoff.values())))

# Compute the option greeks: the sensivities to underlying parameters
greeks = tf.gradients(price, [Spot, σ, K, r])


W0815 12:48:51.746079 140210998773568 deprecation.py:323] From <ipython-input-3-ff0a82dd20e0>:30: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


This is the block where we execute the operations defined above. Before this execution, we must first 
launch a TF session. This is what ensures that the operations will be executed using C++ and CUDA kernels as opposed
to being executed by the Python interpreter.

In [4]:
# Launch the session and initialize global variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# let's print the option price for a few simulations:
for _ in range(10):
    print('Price:       ', sess.run(price))

Price:        4.4799643
Price:        4.4521413
Price:        4.467239
Price:        4.4879293
Price:        4.480185
Price:        4.484444
Price:        4.4778757
Price:        4.4457393
Price:        4.467221
Price:        4.463552


In [5]:
# Notice that the values change slightly at each simulation. That is expected from Monte Carlo simulations.
# To get more precise estimates of the expected payoff, we can averge the results across a large number of
# simulations.
average = np.mean([sess.run(price) for _ in range(1000)])
print(average)

average = np.mean([sess.run(price) for _ in range(1000)])
print(average)

4.466334
4.466745
